Install dependencies

In [1]:
!pip install elasticsearch python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 800.2 kB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 125.6 kB/s eta 0:00:00 0:00:01


In [2]:
import os
from elasticsearch import Elasticsearch, NotFoundError, helpers
import csv
from dotenv import load_dotenv

Load env variables

In [3]:
load_dotenv()

False

Connect to ElasticSearch

In [4]:
es = Elasticsearch(
  "http://elk-and-jupyter-elasticsearch-1:9200",
  basic_auth=("elastic", os.getenv('ELASTIC_PASSWORD')),
)
print(es.info())

{'name': 'elasticsearch', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'SuNirrkfT9-5N6DF_oVYoA', 'version': {'number': '8.10.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'b4a62ac808e886ff032700c391f45f1408b2538c', 'build_date': '2023-10-11T22:04:35.506990650Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Check if CSV is on Elastic

In [7]:
try:
  es.search(
    index='heart-csv-data',
  )
except NotFoundError:
  print('Index Not found. Adding to ElasticSearch...')
  with open('./data/heart.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='heart-csv-data')
    print('OK!')
else:
  print('Data is OK')

Data is OK


Indexing

In [8]:
result = es.index(
 index='heart-csv-data',
 document={
   'age': '69', 
   'sex': '1', 
   'cp': '3', 
   'trestbps': '105', 
   'chol': '240', 
   'fbs': '0', 
   'restecg': '2', 
   'thalach': '154', 
   'exang': '1', 
   'oldpeak': '0.6', 
   'slope': '2', 
   'ca': '0', 
   'thal': 'reversible', 
   'target': '0'
  }
)
print(result)

{'_index': 'heart-csv-data', '_id': 'dtbNp4sBL4LmiW0YnQ9p', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 303, '_primary_term': 1}


Searching

In [9]:
result = es.search(
 index='heart-csv-data',
 query={
    'match': {'age': '58'}
  },
)
print(result)

{'took': 414, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 18, 'relation': 'eq'}, 'max_score': 2.8025408, 'hits': [{'_index': 'heart-csv-data', '_id': 'XNbNp4sBL4LmiW0YOA5T', '_score': 2.8025408, '_source': {'age': '58', 'sex': '0', 'cp': '1', 'trestbps': '150', 'chol': '283', 'fbs': '1', 'restecg': '2', 'thalach': '162', 'exang': '0', 'oldpeak': '1', 'slope': '1', 'ca': '0', 'thal': 'normal', 'target': '0'}}, {'_index': 'heart-csv-data', '_id': 'XdbNp4sBL4LmiW0YOA5T', '_score': 2.8025408, '_source': {'age': '58', 'sex': '1', 'cp': '2', 'trestbps': '120', 'chol': '284', 'fbs': '0', 'restecg': '2', 'thalach': '160', 'exang': '0', 'oldpeak': '1.8', 'slope': '2', 'ca': '0', 'thal': 'normal', 'target': '0'}}, {'_index': 'heart-csv-data', '_id': 'XtbNp4sBL4LmiW0YOA5T', '_score': 2.8025408, '_source': {'age': '58', 'sex': '1', 'cp': '3', 'trestbps': '132', 'chol': '224', 'fbs': '0', 'restecg': '2', 'thalach': '173', 'exa

Updating data

In [10]:
result = es.index(
 index='heart-csv-data',
 document={
  'age': '58',
 })
es.update(
 index='heart-csv-data',
 id=result['_id'], 
 doc={
   'age': '67', 
   'sex': '1', 
   'cp': '3', 
   'trestbps': '105', 
   'chol': '240', 
   'fbs': '0', 
   'restecg': '2', 
   'thalach': '154', 
   'exang': '1', 
   'oldpeak': '0.6', 
   'slope': '2', 
   'ca': '0', 
   'thal': 'reversible', 
   'target': '0'
  }
)
print(es.get(index='heart-csv-data', id=result['_id']))

{'_index': 'heart-csv-data', '_id': 'd9bNp4sBL4LmiW0Yug-f', '_version': 2, '_seq_no': 305, '_primary_term': 1, 'found': True, '_source': {'age': '67', 'exang': '1', 'sex': '1', 'thal': 'reversible', 'chol': '240', 'slope': '2', 'cp': '3', 'trestbps': '105', 'target': '0', 'oldpeak': '0.6', 'thalach': '154', 'fbs': '0', 'ca': '0', 'restecg': '2'}}
